In [ ]:
import pandas as pd 
import sqlite3
import sys
sys.path.insert(0, '..')
import util.util_newssniffer_parsing as up
from importlib import reload
reload(up)

# Get Sentence Diffs

In [ ]:
# sample_ids = nytimes_article_diffs['article_id'].drop_duplicates().sample(2500)
# sample_diffs = nytimes_article_diffs.loc[lambda df: df['article_id'].isin(sample_ids)]
nyt_conn = sqlite3.connect('../data/diffengine-diffs/db/newssniffer-nytimes.db')
sample_diffs_by_article = pd.read_sql("""
    SELECT * from entryversion 
    WHERE entry_id IN (SELECT DISTINCT entry_id from entryversion ORDER BY RANDOM() LIMIT 2500)
""", con=nyt_conn)

sample_ids = sample_diffs_by_article['entry_id'].unique()
sample_diffs_by_article = sample_diffs_by_article.set_index('entry_id')

article_diffs = sample_diffs_by_article.loc[sample_ids[0]].sort_values('version')
version_pairs = article_diffs['version'].pipe(lambda s: list(zip(s[:-1], s[1:])))
sentence_stats, word_stats = [], []
##
for a_id in tqdm(sample_ids):
    a = sample_diffs_by_article.loc[a_id]
    vs = a['version']
    a_by_version = a.set_index('version')

    for v_old, v_new in list(zip(vs[:-1], vs[1:])):
        vars_old, vars_new = up.get_sentence_diff(
            a_by_version.loc[v_old]['summary'],
            a_by_version.loc[v_new]['summary']
        )
        
        doc_changes = up.get_changes(vars_old, vars_new)
        sent_change_pairs = doc_changes['sentences']['changed_sent_pairs']

        sentence_stats.append({
            'num_added_sents': len(doc_changes['sentences']['added_sents']),
            'len_new_doc': len(doc_changes['docs']['new_doc']),
            'num_removed_sents': len(doc_changes['sentences']['removed_sents']),
            'len_old_doc':len(doc_changes['docs']['old_doc']),
            'num_changed_sents': len(doc_changes['sentences']['changed_sent_pairs']),
            'version_nums': (v_old, v_new),
            'vars_old': vars_old,
            'vars_new': vars_new,
            'a_id': a_id,
        })
        
        for sent_pair in doc_changes['sentences']['changed_sent_pairs']:
            s_old, s_new = up.get_word_diffs(*sent_pair)
            word_stats.append({
                'num_words_removed': sum(map(lambda x: x['tag'] == '-', s_old)),
                'num_words_added': sum(map(lambda x: x['tag'] == '+', s_new)),
                'len_old_sent': len(list(filter(lambda x: x['text'] != '' , s_old))),
                'len_new_sent': len(list(filter(lambda x: x['text'] != '', s_new))),
                'version_nums': (v_old, v_new),
                's_old': s_old,
                's_new': s_new,
                'sent_par': sent_pair,
                'a_id': a_id
            })

sentence_stats_df = pd.DataFrame(sentence_stats)
word_stats_df = pd.DataFrame(word_stats)

# Spot Checks

In [ ]:
# corrections
# contributor lines
vo, vn = (
    sentence_stats_df
          .loc[lambda df: df['num_added_sents'] == 10]
          .iloc[2]
          [['vars_old', 'vars_new']]
)

In [ ]:
## possible to-do: explicit positional encoding to spot sentences that were moved

display(HTML(up.html_compare_articles(vo, vn)))

In [ ]:
vot = copy.deepcopy(vo)
vnt = copy.deepcopy(vn)

v1o, v1n = up.merge_all_clusters(vot, vnt, slack=.5)
display(HTML(up.html_compare_articles(v1o, v1n)))

### Saved Spot Checks

In [ ]:
a1 = 'However , her meeting with Mr. Xi was canceled Wednesday morning .'
a2 = 'United States officials said Mr. Xi had cancelled other meetings scheduled for the day so it did not appear that any slight was intended.'
b  = 'The meeting with Mr. Xi was canceled Wednesday morning , although United States officials said they did not think it was intended as a slight because Mr. Xi had also canceled other Wednesday appointments .'


a = 'WASHINGTON – For President Obama, the first test of his second term will come quickly this week when Chuck Hagel and John Kerry, his nominees for the two biggest national-security posts, take critical steps toward winning Senate confirmation.'
b = 'For President Obama, the first test of his second term passed peacefully on Tuesday as his nominee for defense secretary, Chuck Hagel, sat down with the Republican who poses one of his biggest potential hurdles to Senate confirmation, Senator John McCain of Arizona.'

a = 'As Mr. Hagel’s prospects have improved, some analysts say the nominee to watch is Mr. Brennan, the White House counterterrorism adviser Mr. Obama has chosen for the C.I.A..'
b = 'With Mr. Hagel’s prospects improving, some analysts say the nominee to watch is Mr. Brennan, whom Mr. Obama has chosen as a permanent replacement for David H. Petraeus at the C.I.A., who resigned in November after admitting to an extramarital affair.'

a = 'The report did not identify those countries .' 
b = 'Choe Sang - Hun reported from Seoul , and Mark Landler from Washington .'

up.check_subset(b, a, slack=.5)

s1 = a
s2 = b
# lemmatize and get stopwords
s1_lemmas, s2_lemmas = up.get_lemmas(s1), up.get_lemmas(s2)
s1_lemmas, s2_lemmas = up.filter_stopword_lemmas(s1_lemmas), up.filter_stopword_lemmas(s2_lemmas)

In [ ]:
vot = copy.deepcopy(vo)
vnt = copy.deepcopy(vn)
cs = up.cluster_edits(vot, vnt)
c = cs[0]
c = [
    (
        {'text': 'WASHINGTON – For President Obama, the first test of his second term will come quickly this week when Chuck Hagel and John Kerry, his nominees for the two biggest national-security posts, take critical steps toward winning Senate confirmation.', 'tag': '-'},
        {'text': '', 'tag': ' '}
    ),
    (
        {'text': 'They are likely to get very different receptions.', 'tag': '-'},
        {'text': '', 'tag': ' '}
    ),
    (
        {'text': '', 'tag': ' '}, {'text': 'WASHINGTON', 'tag': '+'}
    ),
    (
        {'text': '', 'tag': ' '},
        {'text': 'For President Obama, the first test of his second term passed peacefully on Tuesday as his nominee for defense secretary, Chuck Hagel, sat down with the Republican who poses one of his biggest potential hurdles to Senate confirmation, Senator John McCain of Arizona.', 'tag': '+'}
    ),
    (
        {'text': '', 'tag': ' '},
        {'text': 'By all accounts, the 45-minute one-on-one meeting went well enough for the White House to feel that it has finally turned the tide in favor of Mr. Hagel, whose positions on Israel and Iran have drawn heavy fire from conservatives, pro-Israel groups and other critics.', 'tag': '+'}
    ),
    (
        {'text': '', 'tag': ' '},
        {'text': 'While Mr. McCain did not say how he would vote on Mr. Hagel, he told reporters that he would reserve judgment until after Mr. Hagel’s confirmation hearing on Jan. 31.', 'tag': '+'}
    ),
]

t = up.merge_cluster(c, slack=.5)

# Check Merging

In [1]:
c = [
    (
        {
            'text': 'Breaking a deadlock over the location and timing of new talks on its disputed nuclear program, Iran reached agreement on Tuesday with world powers to resume the stuttering dialogue later this month in Kazakhstan, according to the official IRNA news agency.', 
            'tag': '-'
        }, 
        {
            'text': 'Breaking a deadlock over the location and timing of new talks on its disputed nuclear program, Iran said Tuesday it had reached agreement with world powers to resume the stuttering dialogue later this month in Kazakhstan.', 
            'tag': '+'
        }
    ), (
        {
            'text': 'The agreement to meet there on Feb. 26 came in a telephone conversation between senior officials of Iran’s National Security Council and the European Union, representing the outside powers involved in the talks, IRNA said.', 
            'tag': '-'
        }, 
        {
            'text': 'The agreement to meet there on Feb. 26 came in a telephone conversation between senior officials of Iran’s National Security Council and the European Union, representing the outside powers involved in the talks, Iran’s official Islamic Republic News Agency said.', 
            'tag': '+'
        }
    ), 
    (
        {'text': '', 'tag': ' '}, 
        {'text': '.', 'tag': '+'}
    )
]

In [ ]:
up.merge_cluster(c)

In [ ]:
##############################

s1 = 'In Japan , the transport ministry said it would ground the Dreamliners indefinitely , and in India , the aviation regulator said it had grounded the six aircraft operated by the state - owned carrier Air India , Reuters reported .'
s2 = 'In India , the aviation regulator grounded all six of the 787s operated by the state - owned carrier Air India .'
s3 = 'In Japan the transport ministry issued a formal order to ground all 787s indefinitely, until concerns over the aircraft’s battery systems are resolved.'
up.check_subset(s1, s3)
up.check_subset(b, a)

#############################
slack = .3
s1 = up.merge_sents_list(s1) if isinstance(s1, list) else s1
s2 = up.merge_sents_list(s2) if isinstance(s2, list) else s2
# lemmatize and get stopwords
s1_lemmas, s2_lemmas = up.get_lemmas(s1), up.get_lemmas(s2)
s1_lemmas, s2_lemmas = up.filter_stopword_lemmas(s1_lemmas), up.filter_stopword_lemmas(s2_lemmas)
### check match.
matches = sum(map(lambda word: word in s1_lemmas, s2_lemmas))
matches >= (len(s2_lemmas) * (1 - slack))

(len(s2_lemmas) * (1 - slack))

sum(map(lambda word: word in s1_lemmas, s2_lemmas))

sum(map(lambda word: word in s2_lemmas, s1_lemmas))

(len(s2_lemmas) * (1 - slack))

(len(s1_lemmas) * (1 - slack))

############################
a = 'North Korea said on Tuesday that it will both restart its old nuclear reactor, which had provided plutonium fuel for its nuclear weapons until it was mothballed in 2007, and rev up its uranium enrichment.'
b = 'North Korea said on Tuesday that it will put all its nuclear facilities — including its operational uranium-enrichment program and its reactors mothballed or under construction — to use in expanding its nuclear weapons arsenal, raising the stakes in the escalating standoff with the United States and its allies.'

### get all text (might be a list).
s1 = merge_sents_list(a) if isinstance(a, list) else a
s2 = merge_sents_list(b) if isinstance(b, list) else b
s1, s2 = s2, s1
# lemmatize and get stopwords
s1_lemmas, s2_lemmas = up.get_lemmas(s1), up.get_lemmas(s2)
s1_lemmas, s2_lemmas = filter_stopword_lemmas(s1_lemmas), filter_stopword_lemmas(s2_lemmas)
### check match.
matches = list(map(lambda word: word in s1_lemmas, s2_lemmas))

sum(matches)

len(s2_lemmas)

# Run Diff Generator Ourselves

In [ ]:
(sentence_stats_df
          .loc[lambda df: df['num_added_sents'] == 1]
          .iloc[24])

a, b = sample_diffs_by_article.loc[593406].loc[lambda df: df['version'].isin((3,4))]['summary']

reload(up)

a_old_sents = up.split_sents(a)
a_new_sents = up.split_sents(b)
a_old_sents = list(filter(up.filter_sents, a_old_sents))
a_new_sents = list(filter(up.filter_sents, a_new_sents))

vo, vn = (sentence_stats_df
          .loc[lambda df: df['num_added_sents'] == 1]
          .iloc[21]
          [['vars_old', 'vars_new']]
         )

(sentence_stats_df
          .loc[lambda df: df['num_added_sents'] == 1]
          .iloc[21])

a, b = sample_diffs_by_article.loc[592103].loc[lambda df: df['version'].isin((3, 4))]['summary']

a_old_sents = up.split_sents(a)
a_new_sents = up.split_sents(b)
a_old_sents = list(filter(up.filter_sents, a_old_sents))
a_new_sents = list(filter(up.filter_sents, a_new_sents))
# get_list_diff(a_old_sents, a_new_sents)

reload(up)

vo, vn = up.get_list_diff(a_old_sents, a_new_sents)

display(HTML(up.html_compare_articles(vo, vn)))

vot = copy.deepcopy(vo)
vnt = copy.deepcopy(vn)

display(HTML(up.html_compare_articles(*merge_all_clusters(vo, vn))))



(sentence_stats_df
          .loc[lambda df: df['num_added_sents'] == 1]
          .iloc[4]
)
a_old, a_new = sample_diffs_by_article.loc[564771].loc[lambda df: df['version'].isin((4,5))]['summary']

# Glove

In [ ]:
import numpy as np 
import spacy
from scipy.spatial.distance import cosine
nlp = spacy.load("en_core_web_lg", disable=["tagger", "ner"])  # just the parser

def get_all_glove_vecs():
    glove = open('../data/glove.6B/glove.6B.300d.txt').read()
    glove_lines = glove.split('\n')
    glove_lines = list(map(lambda x: x.split(), glove_lines))
    glove_words = list(map(lambda x: x[0], glove_lines[:-1]))
    glove_vecs = list(map(lambda x: np.array(list(map(float, x[1:]))), glove_lines[:-1]))
    return pd.Series(glove_vecs, index=glove_words)

def get_sentence_glove_vec(sent):
    sent_vec = []
    for word in nlp(sent):
        word = word.text.lower()
        if word in glove:
            sent_vec.append(glove[word])
    return np.array(sent_vec).mean(axis=0)

glove = get_all_glove_vecs()

In [ ]:
ao = 'An earlier version of a picture caption with this article incorrectly described a ceremony being performed by a Navy chaplain for an Air Force sergeant and his partner at a military base in New Jersey.'
an = 'A picture caption on Thursday with an article about gay, lesbian and bisexual members of the military service described incorrectly a ceremony being performed by a Navy chaplain for an Air Force sergeant and his partner at a military base in New Jersey.'
print(ao)
print()
print(an)
print('ao subset an? %s.' % check_subset(ao, an))
print('an subset ao? %s.' % check_subset(an, ao))
print('------------------------------------------')

ao = 'In a microblog post on Friday, Ran Yunfei, a sharp-tongued writer persecuted for his pro-democracy writings, said he now hoped that Mr. Mo would stand up more boldly for those who have dared to speak truth to power.'
an = 'Ran Yunfei, a sharp-tongued writer persecuted for his pro-democracy views, said he was heartened by Mr. Mo’s comments but doubted that he would become a crusader for human rights and free expression.'
print(ao)
print()
print(an)
print('ao subset an? %s.' % check_subset(ao, an))
print('an subset ao? %s.' % check_subset(an, ao))
print('--------------------------------------------')

ao = 'He relished his work on the Judiciary Committee.'
an = 'He relished the decades he spent on the Judiciary Committee.'
print(ao)
print()
print(an)
print('ao subset an? %s.' % check_subset(ao, an))
print('an subset ao? %s.' % check_subset(an, ao))
print('------------------------------------------')

ao = 'Mr. Samaha’s arrest was widely viewed as evidence of a Syrian plot to spread sectarian mayhem in Lebanon.'
an = 'Mr. Samaha’s arrest was widely seen as part of Lebanese government efforts to prevent the spread of sectarian mayhem in the country.'
print(ao)
print()
print(an)
print('ao subset an? %s.' % check_subset(ao, an))
print('an subset ao? %s.' % check_subset(an, ao))
print('------------------------------------------')

ao = 'An earlier version of this article misstated the distance between Alpha Centauri A and Alpha Centauri B, the two main stars in the triple-star Alpha Centauri system.	'
an = 'An article on Wednesday about a new planet discovered in the Alpha Centauri system misstated the distance between Alpha Centauri A and Alpha Centauri B, the two main stars in the triple-star system.'
print(ao)
print()
print(an)
print('ao subset an? %s.' % check_subset(ao, an))
print('an subset ao? %s.' % check_subset(an, ao))
print('------------------------------------------')

ao = 'On Tuesday night, they were here in Charlotte without being here.'
an = 'On Thursday night, they appeared in a video and in person before and after their father’s speech, but on Tuesday night, they were here in Charlotte without being here.'
print(ao)
print()
print(an)
print('ao subset an? %s.' % check_subset(ao, an))
print('an subset ao? %s.' % check_subset(an, ao))
print('------------------------------------------')

ao = 'President Obama’s first postelection overseas trip will be this month to Asia, where he is to make a historic visit to Myanmar as it moves toward democracy and reinforce his desire to reorient American foreign policy more toward the Pacific during his second term.'
an = 'President Obama will make Asia his first overseas destination since his re-election, with a trip this month that is to include a historic visit to Myanmar and underscore his desire to reorient American foreign policy more toward the Pacific during his second term.'
print(ao)
print()
print(an)
print('ao subset an? %s.' % check_subset(ao, an))
print('an subset ao? %s.' % check_subset(an, ao))
print('------------------------------------------')

In [ ]:
ao_vec, an_vec = get_sentence_glove_vec(ao), get_sentence_glove_vec(an)
cosine(ao_vec, an_vec)

ao_vec, an_vec = get_sentence_glove_vec(ao), get_sentence_glove_vec(an)
cosine(ao_vec, an_vec)

a_rand_1 = '“I get in front of the Marines as often as I can, as long as I can get away from Washington, and I’ll be honest with you, I don’t even get a question,” the Marine commandant, James F. Amos, said at the National Press Club last month.'
a_rand_2 = "Aubrey Sarvis, an Army veteran and the executive director of the Servicemembers Legal Defense Network, said in an interview that after the female officers contacted his organization, the Pentagon investigated and the squadron commander and the sergeant major were relieved of their jobs and forced to retire."

ar_1_vec, ar_2_vec = get_sentence_glove_vec(a_rand_1), get_sentence_glove_vec(a_rand_2)
cosine(ar_1_vec, ar_2_vec)

print('ao subset an? %s.' % check_subset(a_rand_1, a_rand_2))
print('an subset ao? %s.' % check_subset(a_rand_2, a_rand_1))

## cases where added sentence is a subset of removed words of other sentences
## cases where added sentence is semantically equivalent to removed sentence
sentence_stats_df.loc[lambda df: df['num_added_sents'] == 1].iloc[0][['version_nums', 'a_id']]
list(filter(lambda x: x['tag'] == '+', vn))
list(map(lambda x: x['text'], vo))